# Phase 3: Feature Engineering for Business Signals

## 1. Business Problem Statement
Raw transaction data does not directly capture customer value, demand patterns,
or operational risk. To enable forecasting and segmentation we must engineer features that represent meaningful business
behavior rather than individual transactions.

## 2. Why This Matters to the Business
Well-designed features allow the business to anticipate customer needs, forecast
demand accurately, identify high-value customers, and make proactive decisions.
Poor feature design leads to fragile models and misleading insights.

In [1]:

import pandas as pd
from pathlib import Path

df = pd.read_csv(
    "../data/processed/cleaned_data.csv",
    parse_dates=["Order Date", "Ship Date"]
)

df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
# ======================================================
# 1. TIME-BASED FEATURES
# ======================================================

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['Order Date'] = pd.to_datetime(
        df['Order Date'],
        format='mixed',
        dayfirst=False
    )

    df['order_year'] = df['Order Date'].dt.year
    df['order_month'] = df['Order Date'].dt.month
    df['order_quarter'] = df['Order Date'].dt.quarter
    df['order_dayofweek'] = df['Order Date'].dt.dayofweek

    return df
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [6]:
# ======================================================
# 2. PROFIT & DISCOUNT FEATURES
# ======================================================

def add_profit_discount_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['profit_margin'] = df.apply(
        lambda x: x['Profit'] / x['Sales'] if x['Sales'] > 0 else 0,
        axis=1
    )

    df['is_discounted'] = (df['Discount'] > 0).astype(int)

    return df
df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [8]:
# ======================================================
# 3. CUSTOMER AGGREGATION FEATURES
# ======================================================

def create_customer_aggregates(df: pd.DataFrame) -> pd.DataFrame:
    customer_agg = (
        df
        .groupby('Customer ID')
        .agg(
            Sales=('Sales', 'sum'),
            Profit=('Profit', 'sum'),
            Discount=('Discount', 'mean'),
            Order_ID=('Order ID', 'nunique')
        )
        .reset_index()
    )

    customer_agg.rename(columns={
        'Sales': 'total_sales',
        'Profit': 'total_profit',
        'Discount': 'avg_discount',
        'Order_ID': 'num_orders'
    }, inplace=True)

    return customer_agg


In [9]:
# ======================================================
# 4. RFM FEATURE ENGINEERING
# ======================================================

def create_rfm_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Order Date'] = pd.to_datetime(df['Order Date'])

    snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)

    rfm = (
        df
        .groupby('Customer ID')
        .agg(
            Recency=('Order Date', lambda x: (snapshot_date - x.max()).days),
            Frequency=('Order ID', 'nunique'),
            Monetary=('Sales', 'sum')
        )
        .reset_index()
    )

    return rfm


In [10]:
# ======================================================
# 5. MASTER FEATURE PIPELINE
# ======================================================

def build_feature_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df = add_time_features(df)
    df = add_profit_discount_features(df)
    return df


In [11]:
# ======================================================
# 6. APPLY FEATURE PIPELINE
# ======================================================

df = build_feature_dataset(df)
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sales,Quantity,Discount,Profit,order_year,order_month,order_quarter,order_dayofweek,profit_margin,is_discounted
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,261.9600,2,0.00,41.9136,2016,11,4,1,0.1600,0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,731.9400,3,0.00,219.5820,2016,11,4,1,0.3000,0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,14.6200,2,0.00,6.8714,2016,6,2,6,0.4700,0
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,957.5775,5,0.45,-383.0310,2015,10,4,6,-0.4000,1
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,22.3680,2,0.20,2.5164,2015,10,4,6,0.1125,1


In [12]:
# ======================================================
# 7. CUSTOMER-LEVEL DATASETS
# ======================================================

customer_agg_df = create_customer_aggregates(df)
rfm_df = create_rfm_features(df)

customer_agg_df.head(), rfm_df.head()


(  Customer ID  total_sales  total_profit  avg_discount  num_orders
 0    AA-10315     5563.560     -362.8825      0.090909           5
 1    AA-10375     1056.390      277.3824      0.080000           9
 2    AA-10480     1790.512      435.8274      0.016667           4
 3    AA-10645     5086.935      857.8033      0.063889           6
 4    AB-10015      886.156      129.3465      0.066667           3,
   Customer ID  Recency  Frequency  Monetary
 0    AA-10315      185          5  5563.560
 1    AA-10375       20          9  1056.390
 2    AA-10480      260          4  1790.512
 3    AA-10645       56          6  5086.935
 4    AB-10015      416          3   886.156)

In [13]:
# ======================================================
# 8. SAVE FEATURED DATASETS
# ======================================================

output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv("../data/processed/featured_data.csv", index=False)
customer_agg_df.to_csv("../data/processed/customer_rfm.csv", index=False)

print("✅ Featured datasets saved successfully")


✅ Featured datasets saved successfully


In [ ]:
# ======================================================
# 9. EXECUTIVE METRICS 
# ======================================================

monthly_exec_metrics = (
    df
    .assign(
        Order_Month=df["Order Date"]
        .dt.to_period("M")
        .dt.to_timestamp()
    )
    .groupby("Order_Month")
    .agg(
        total_sales=("Sales", "sum"),
        total_profit=("Profit", "sum"),
        avg_discount=("Discount", "mean"),
        total_quantity=("Quantity", "sum")
    )
    .reset_index()
)


In [15]:
# ======================================================
# 10. EXECUTIVE KPIs (STREAMLIT READY)
# ======================================================

monthly_exec_metrics["profit_margin"] = (
    monthly_exec_metrics["total_profit"] /
    monthly_exec_metrics["total_sales"]
)

monthly_exec_metrics["sales_mom"] = (
    monthly_exec_metrics["total_sales"].pct_change()
)

monthly_exec_metrics["profit_mom"] = (
    monthly_exec_metrics["total_profit"].pct_change()
)

monthly_exec_metrics = (
    monthly_exec_metrics
    .dropna()
    .reset_index(drop=True)
)

monthly_exec_metrics.head()


,Order_Month,total_sales,total_profit,avg_discount,total_quantity,profit_margin,sales_mom,profit_mom
0,2014-02-01,4519.8920,862.3084,0.176087,159,0.190781,-0.682523,-0.648065
1,2014-03-01,55691.0090,498.7299,0.167516,585,0.008955,11.321314,-0.421634
2,2014-04-01,28295.3450,3488.8352,0.110000,536,0.123301,-0.491923,5.995440
3,2014-05-01,23648.2870,2738.7096,0.155328,466,0.115810,-0.164234,-0.215007
4,2014-06-01,34595.1276,4976.5244,0.172000,521,0.143850,0.462902,0.817106


In [17]:
# ======================================================
# 11. SAVE EXECUTIVE METRICS
# ======================================================

monthly_exec_metrics.to_csv(
    "../data/processed/executive_metrics_monthly.csv",
    index=False
)

print("✅ executive_metrics_monthly.csv generated correctly")


✅ executive_metrics_monthly.csv generated correctly
